In [1]:
import requests
import os
from bs4 import BeautifulSoup
import re

import urllib.parse

In [2]:
siteUrl = "http://www.collectorcarads.com/advancedsearch.asp"
print(siteUrl)

http://www.collectorcarads.com/advancedsearch.asp


In [3]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=siteUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

In [4]:
from random import randint

# Get the year links
tds = tagPageSoup.find_all('td', class_='makelist')
yearLinkSoup = tds[2]
yearLinks = yearLinkSoup.find_all("a", class_="sidelinks")
print(len(yearLinks))

yearLink = yearLinks[randint(0,len(yearLinks)-1)]
url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', yearLink['href']))
print(url)

carPage = requests.get(url=url)
carTdSoup = BeautifulSoup(carPage.content, 'lxml')
carTds = carTdSoup.find_all("td", class_="lines")
print(len(carTds))
carLinks = carTds[1].find_all("a")
print(len(carLinks))

carLink = carLinks[randint(0,len(carLinks)-1)]
carLinkUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', carLink['href']))
print(carLinkUrl)

112
http://www.collectorcarads.com/year/1975
2
39
http://www.collectorcarads.com/Mercedes-450SL/79258


In [5]:
dealerUrl = "http://www.collectorcarads.com/Ford-Thunderbird/78172"
privateUrl = "http://www.collectorcarads.com/Other-Ache-Freres/78965"
page = requests.get(url=privateUrl)
soup = BeautifulSoup(page.content, 'lxml')

dealer_css = "td.adtext > p:nth-of-type(1) > a"
dealer_link_text = "View all the ads by this advertiser"
dealer_link = soup.select(dealer_css)
is_dealer = True
if len(dealer_link):
    match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
    if match:
        is_dealer = True
    else:
        is_dealer = False
else:
    is_dealer = False
if not is_dealer:
    print("Private Seller")
else:
    print("Dealer")

model = soup.find_all("h1")
if len(model):
    model = model[0].get_text().strip()
else:
    model = "Unknown"
print(model)

price = soup.find_all("span", class_="pricebold")
if len(price):
    price = price[0].get_text().strip()
else:
    price = "Unknown"
print(price)

phone = soup.select("span.adtext > strong")
if len(phone):
    phone = phone[0].get_text().strip()
print(phone)

Private Seller
1902 Other Ache Freres
$115,000
909-436-8814


In [6]:
dealer_css = "td.adtext > p:nth-of-type(1) > a"
dealer_link_text = "View all the ads by this advertiser"

In [7]:
# Retrieve the page with tag results and set it up to be scraped
tagPage = requests.get(url=siteUrl)
tagPageSoup = BeautifulSoup(tagPage.content, 'lxml')

# Get the year column
tds = tagPageSoup.find_all('td', class_='makelist')
yearLinkSoup = tds[2]
yearLinks = yearLinkSoup.find_all("a", class_="sidelinks")

n = 0
for yearLink in yearLinks:
    url = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', yearLink['href']))
    #print('url: ' + url)
    carPage = requests.get(url=url)
    carTdSoup = BeautifulSoup(carPage.content, 'lxml')
    carTds = carTdSoup.find_all("td", class_="lines")
    #print('carTds: ' + str(len(carTds)))
    if len(carTds):
        carLinks = carTds[1].select("p > b:nth-of-type(1) > font > a")
        #print('carLinks: ' + str(carLinks))
        for carLink in carLinks:
            carLinkUrl = urllib.parse.urljoin(siteUrl, re.sub(' ', '%20', carLink['href']))
            #print('carLinkUrl: ' + carLinkUrl)
            try:
                page = requests.get(url=carLinkUrl)
                soup = BeautifulSoup(page.content, 'lxml')

                dealer_link = soup.select(dealer_css)
                #print('dealer_link: ' + str(dealer_link))
                is_dealer = True
                if len(dealer_link):
                    match = re.search(dealer_link_text, dealer_link[0].get_text().strip())
                    if match:
                        is_dealer = True
                    else:
                        is_dealer = False
                else:
                    is_dealer = False
                if not is_dealer:
                    suffix = "Private Seller"
                else:
                    suffix = "Dealer"
                #print('suffix: ' + suffix)

                model = soup.find_all("h1")
                if len(model):
                    model = model[0].get_text().strip()
                else:
                    model = "Unknown"
                #print('model: ' + model)
                model = model + " (" + suffix + ")"

                price = soup.find_all("span", class_="pricebold")
                if len(price):
                    price = price[0].get_text().strip()
                else:
                    price = "Unknown"
                #print('price: ' + price)

                phone = soup.select("span.adtext > strong")
                if len(phone):
                    phone = phone[0].get_text().strip()
                if len(phone):
                    #print('phone: ' + str(phone))
                    bigFile = open('collectorcarads.txt', 'a')
                    bigFile.write(model + '\t' + price + '\t' + phone + '\n')
                    n = n + 1
                    bigFile.close()
            except Exception as e:
                print(str(e))
print(str(n) + " phone numbers written to file")

1560 phone numbers written to file
